In [4]:
import numpy as np
import datetime
import pandas as pd
import cv2

from PIL import Image
import os
from cv2 import *

In [5]:
#used to set new attributes in the csv data
def setRec(id,name):
    
    #importing the data frame from csv file
    x=pd.read_csv("attendance.csv")
    y=pd.read_csv("roll.csv")
    
    #adding a student and roll no sheet
    a=[id,name]
    #
    m=y.append({"Roll":id,"Name":name},ignore_index=True)
    #writting back to csv file
    m.to_csv("roll.csv",index=False)
    
    #adding a student in attendance sheet
    x.update({id:[]})
    #writting back to csv file
    x.to_csv("attendance.csv",index=False)

In [6]:
#associating image with id
def getImagesWithId(path):
    
    #setting the image path
    imagePaths=[os.path.join(path,f) for f in os.listdir(path)]
    
    #initializing the values
    faces=[]
    IDs=[]
    
    #iterating for every image in the directory
    for imagePath in imagePaths:
        
        #getting the picture data from path and converting it into 8 bit black and white (L)
        faceImg=Image.open(imagePath).convert("L")
        
        #converting the pic into a numpy array in integer data type (8 bit)
        faceNp=np.array(faceImg,"uint8")
        
        #getting back the student id from the photo name
        ID=int(os.path.split(imagePath)[-1].split(".")[1])
        
        #appending the face matrix in the list
        faces.append(faceNp)
        
        #appending the id values in the list
        IDs.append(ID)
        
        #opening the training window
        cv2.imshow("training",faceNp)
        cv2.waitKey(10)
    
    #returning the id and face list
    return np.array(IDs), faces

In [7]:
def trainMachine():
    
    ##getting the cascade information from openCV
    recognizer=cv2.face.LBPHFaceRecognizer_create()
    
    #setting the dataset path
    path="dataset/"

    #geting the id and associated images
    Ids,faces=getImagesWithId(path)
    
    #traing with image data and ralting with id
    recognizer.train(faces,Ids)
    
    #saving the training infromation in location
    recognizer.save("recognizer/trainingData.yml")
    
    #closing the training window
    cv2.destroyWindow("training")

In [8]:
#used to collect the sample images i.e. used to train the machine
def train():
    #getting the cascade information from openCV
    faceDetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    #getting the image source from web camera
    cam=cv2.VideoCapture(0)
    
    #initializing sample no, It is used to name the sample photos
    sampleNum=0
    
    #getting the dataframe from csv
    y=pd.read_csv("roll.csv")
    
    #setting the id value as the no of columns in the dataframe
    #to assign new student we have to increase the value of id
    id=str(y.shape[0])
    
    #print the id value of the new student
    print(id)
    
    #getting the student name
    name=input("Enter student name : ")
    
    #updating the student id and name in the csv
    setRec(id,name)

    #running the image capturing window
    while(True):
        
        #reading the frame
        rett,img=cam.read()
        
        #convering the color image into grayscale as the cascade only understand grayscale value
        gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        #histogram equalizing of the image
        img_hist = cv2.equalizeHist(gray)
        
        #getting the multiscale value
        faces=faceDetect.detectMultiScale(img_hist,1.3,5)
        
        #collecting the sample images and putting a rectangle frame on face 
        for(x,y,w,h) in faces:
            
            #increasing the sample number with each iteration
            sampleNum=sampleNum+1
            
            #storing the sample images
            cv2.imwrite("dataset/User."+str(id)+"."+str(sampleNum)+".jpg",img_hist[y:y+h,x:x+w])
            
            #showing the rectangle
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
            cv2.waitKey(100)
        
        #starting the webcam window
        cv2.imshow("face",img)
        
        #putting regular interval between two samples to take diverse images
        cv2.waitKey(2)
        
        #stopping the capturing process after collection of 30 successful face samples
        if (sampleNum>30):
            break
            
     #closing camera window
    cam.release()
    cv2.destroyWindow("face")
    cv2.waitKey(1)
    
    #training the machine
    trainMachine()
 

In [9]:
def exportStu(x):
    
    #get the previous data
    att=pd.read_csv("attendance.csv")
    #adding the data to the data frame
    m=att.append(x,ignore_index=True)
    #writting back the updated data to the csv databse
    m.to_csv("attendance.csv",index=False)
    
    
    #fromatting the dictionary to fir the data frame
    for i in x:
        x[i]=[x[i]]
    
    #converting the dictionary to dataframe
    x=pd.DataFrame.from_dict(x)
    #printing today's attendance report

    return m,x

In [10]:
def update(x,y):
    #creating a blank dictionary to update the data frame
    attend={}
    
    #putting the recognised persons (images) in the attendance register
    for i in y:

        #x=pd.read_csv("attendance.csv")   
        d = datetime.datetime.today().strftime("%m/%d/%Y")
        attend["Date"] = d
        
        #as index is same as roll no
        j=y.index(i)
        
        #1: present 0: absent
        if i in x:
            attend[str(j)]=1
        else:
            attend[str(j)]=0
            
    print("Updation Complete\n")
    
    #returning today's data as well as historical data
    return exportStu(attend)

In [11]:
#this will start the process of attendance taking
def start():
    #getting the cascade information from openCV
    faceDetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    #getting the image source from web camera
    cam=cv2.VideoCapture(0)
    rec=cv2.face.LBPHFaceRecognizer_create()
    
    #declaring font of the text in the frame
    font=cv2.FONT_HERSHEY_COMPLEX
    
    #get the student list
    y=pd.read_csv("roll.csv")
    stuList=y["Name"].values.tolist()

    #reading the training data
    rec.read("recognizer/trainingData.yml")
    
    #initializing id value
    id=0
    
    #initializing an empty set to contain the student name 
    a=set()
    
    
    #running the image capturing window
    while(True):
        
        #reading the frame
        rett,img=cam.read()
        
        #convering the color image into grayscale as the cascade only understand grayscale value
        gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        #getting the multiscale value
        faces=faceDetect.detectMultiScale(gray,1.3,5)
        
        #putting a rectangle frame on face 
        for(x,y,w,h) in faces:
            
            #plotting the rectangle
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
            #predicting student id from the face in rectangle
            id,conf=rec.predict(gray[y:y+h,x:x+w])
            
            #getting student name from student id
            id=stuList[id]
            
            #putting the name in the rectangle
            cv2.putText(img,str(id),(x,y+h),font,0.55,(0,255,0),1)
        
        #starting webcam
        cv2.imshow("Face",img)
        
        #updating the attened student list
        a.add(id)
        
        #stopping camera
        if (cv2.waitKey(1)==ord("q")):
            break
    
    #closing camera window
    cam.release()
    cv2.destroyWindow("Face")
    
    #printing attended student list
    print("Student Presnt :",a)
    
    #returning today's data as well as historical data
    return update(a,stuList)

In [12]:
def sheet():
    att = pd.read_csv("attendance.csv")
    return att

## Quick Instruction

**print train() for new student**<br>
**print start() for attendence**<br>
**print sheet() to print attendence sheet**

In [43]:
from tkinter import *
root = Tk()


topFrame = Frame(root)
topFrame.pack()
bottomFrame = Frame(root)
bottomFrame.pack(side=BOTTOM)

button1 = Button(topFrame, text="sheet", command = sheet)
button2 = Button(topFrame, text="New comer", command= train)
button3 = Button(bottomFrame, text="Attendence", command=start)


button1.pack(padx=5, pady=10,side=BOTTOM)
button2.pack(padx=5, pady=10,side=BOTTOM)
button3.pack(padx=5, pady=10,side=BOTTOM)

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\HP\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-43-2da618e206bf>", line 7, in helloCallBack
    tkinter.messagebox.showinfo(att)
  File "C:\Users\HP\Anaconda3\lib\tkinter\messagebox.py", line 83, in showinfo
    return _show(title, message, INFO, OK, **options)
  File "C:\Users\HP\Anaconda3\lib\tkinter\messagebox.py", line 70, in _show
    if title:   options["title"] = title
  File "C:\Users\HP\Anaconda3\lib\site-packages\pandas\core\generic.py", line 1478, in __nonzero__
    .format(self.__class__.__name__))
ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
